In [ ]:
#To set up your workspace, use your specified filepath
#Be sure to change all directory and file names to your own

In [ ]:
#Permit Google Colab to access files in google drive
#Can either (1) mount using the following line of code or (2) set manually in left hand 'Files' panel if using Google Colaboratory
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Install packages 
!pip install geopandas
!pip install rasterio
!pip install rasterstats
!pip install rioxarray

In [ ]:
#Import required libraries
##Must be run each time session is started
import glob 
import os 
import ogr
import gdal, gdalconst 

import rasterio as rio
from rasterio.plot import show
import rasterstats 
import rioxarray
import xarray
from shapely.geometry import mapping

import numpy as np 
import geopandas as gpd 
import matplotlib.pyplot as plt

# Pre 2016 Monitoring

It is recommended to make two folders; Pre 2016 and Post 2016

In [ ]:
filepath_moni = '/content/drive/my_file_path/' ## File path of all data

### Here define filename needed for following analysis
Project_shp = os.path.join(filepath_moni, 'ProjectBoundary.shp') ## file path of boundary shapefile (Change ProjectBoundary accordingly)
NAFD_file = os.path.join(filepath_moni,'NAFD_MD_1984_2016_albers_30m-001.tif') ##NAFD data, can be found in README, similar function to GFW
AGB_prefix = os.path.join(filepath_moni,'AGB_Years/band') ## file prefix of AGB files which have original projection

yrs = np.arange(2010,2016+1) ## years to calculate gain and loss; change if using a different time scale


## Load Project shp as gpd
Project_gpd = gpd.read_file(Project_shp)
## Extract geometry for following clipping
Project_geo = Project_gpd.geometry.apply(mapping)


## Load NAFD
NAFD_da = rioxarray.open_rasterio(NAFD_file)
NAFD_da = NAFD_da.rename({'band':'time'}) ## change dimension name of 'band' to 'time'
NAFD_da['time'] = range(1984,2016+1) ## assign time
NAFD_da = NAFD_da.sel(time=yrs) ## select bands corresponding to yrs
NAFD_da = NAFD_da.rio.clip(Project_geo,crs=Project_gpd.crs,drop=True) ## clip NAFD to project boundary


## Load AGB for each of yrs
AGB_da_list = [] # defining each raster's year 
for yr in yrs:
  tmp_agb_da = rioxarray.open_rasterio('%s_%d.tif' % (AGB_prefix,yr)) ## Loading AGB for given year
  tmp_agb_da = tmp_agb_da.where(tmp_agb_da >= -100) ## mask out no data value
  tmp_agb_da = tmp_agb_da.rename({'band':'time'}) ## change dimension name of 'band' to 'time'
  tmp_agb_da = tmp_agb_da.assign_coords({'time':[yr]}) ## assign time
  tmp_agb_da = tmp_agb_da.interp(x=NAFD_da['x'].data,y=NAFD_da['y'].data) ## interpolate AGB to same coordinates of NAFD so that theya are align
  tmp_agb_da = tmp_agb_da.rio.clip(Project_geo,crs=Project_gpd.crs,drop=True) ## clip NAFD to project domain
  AGB_da_list.append(tmp_agb_da) ## append annual AGB to a list which will be concatenated latter
  del tmp_agb_da

AGB_ts_da = xr.concat(AGB_da_list,dim='time') ## concatenate agb of each individual years
# 3 dimensions -> (time, x, y) with same coords as NAFD 

AGB_ts_da = AGB_ts_da.where(AGB_ts_da>=-100) ## re-mask, as clipping step change masked pixesl to some invalid values.
del AGB_da_list ## remove temporary list to save RAM


## for loop for calculating annual C change
for yr in yrs:
  if yr == yrs[0]: ## skip the first year as it does not have previous year
    continue

  ## calculate AGB difference/growth from last to current year
  AGB_diff = AGB_ts_da.sel(time=yr) - AGB_ts_da.sel(time=yr-1) # can select specific time instead of indexing 

  ## load NAFD in last year and current year
  nafd_cur_yr = NAFD_da.sel(time=yr)
  nafd_lst_yr = NAFD_da.sel(time=yr-1)

  ## define mask of pixels disburbed from last to current year
  loss_mask = (nafd_lst_yr == 5) & (nafd_cur_yr == 7)
  loss_mask = loss_mask | ((nafd_lst_yr == 5) & (nafd_cur_yr >= 100))

  ## define mask of pixels recovered from last to current year
  gain_mask = (nafd_lst_yr == 7) & (nafd_cur_yr == 5)
  gain_mask = (nafd_lst_yr >= 100) & (nafd_cur_yr == 5)

  ## define mask of pixels without disturbance and recovery
  unchange_mask = ~loss_mask & ~gain_mask

  ## calculate total carbon changes over given mask
  c_loss_ttl = AGB_diff.where(loss_mask).sum().data*30*30/1e3
  c_gain_ttl = AGB_diff.where(gain_mask).sum().data*30*30/1e3
  c_unchange_ttl = AGB_diff.where(unchange_mask).sum().data*30*30/1e3

  # dont need to reproject b/c pixel size is meter based 

  # print('%d %d %d %d %d' % (yr, unchange_mask.sum()+loss_mask.sum()+gain_mask.sum(), unchange_mask.sum(), loss_mask.sum(), gain_mask.sum()))
  print('%d' % yr)
  print('   loss %.2f Mg C, %d 30x30m pixels' % (c_loss_ttl,loss_mask.sum().data))
  print('   regrowth %.2f Mg C, %d 30x30m pixels' % (c_gain_ttl,gain_mask.sum().data)) # regrowth 
  print('   continued growth %.2f Mg C, %d 30x30m pixels' % (c_unchange_ttl,unchange_mask.sum().data)) # continued growth 
  print('\n')


# Post 2016 Monitoring

In [ ]:
# STEP 1 - Open and load shapefile of specific property 

In [ ]:
filepath = '/content/drive/my_file_path'
boundary_shp = os.path.join(filepath, 'ProjectBoundary.shp') ## CHANGE to your boundary shapefile
boundary_gpd = gpd.read_file(boundary_shp) # this layer is for CRS MATCHING
boundary_geo = boundary_gpd.geometry.apply(mapping) # this layer is for CLIPPING

### feel free to change variable names to something that makes more sense to you (ex. boundary_shp)
### if you do change the variable name, make sure you're consistent (ex. boundary_shp -> boundary_gpd -> boundary_geo)

In [ ]:
# STEP 2 - Preprocessing 

In [ ]:
# STEP 2a - Reproject GFW loss layer to match coordinate reference system (CRS) of carbon stock raster layer
stock_17_file = os.path.join(filepath, 'AGB_Years/my_file_path/band_2017.tif')
stock_17 = rioxarray.open_rasterio(stock_17_file).astype(np.float32)
stock_17 = stock_17.where(stock_17 >= -100)

### any carbon monitoring layer works since they all have same CRS, here we use the 2017 stock

GFW_loss_raw_file = os.path.join(filepath, 'Hansen_GFC-2021-v1.9_lossyear_40N_080W.tif') #GFW data, can be found in README, similar function to NAFD
GFW_loss_raw_da = rioxarray.open_rasterio(GFW_loss_raw_file)
GFW_loss_raw_da = GFW_loss_raw_da.sel(x=slice(-80,-72),y=slice(40,35)).load()
GFW_loss_reproj_da = GFW_loss_raw_da.rio.reproject_match(stock_17) 
GFW_loss_reproj_da = GFW_loss_reproj_da.where(GFW_loss_reproj_da<255)

# STEP 2b - Clip each layer to match extent of property 
GFW_loss_reproj_da = GFW_loss_reproj_da.rio.clip(boundary_geo,crs=boundary_gpd.crs) ## CHANGE to name of property variable -> boundary_geo and boundary_gpd are TWO different things, be mindful (see above)
GFW_loss_reproj_da = GFW_loss_reproj_da.where(GFW_loss_reproj_da<255).where(GFW_loss_reproj_da>0)
GFW_loss_reproj_da.rio.to_raster(os.path.join(filepath, 'LossPreprocess/ProjectBoundary_loss_preprocess.tif')) ## RENAME file with boundary name (i.e., replace 'ProjectBoundary' in 'ProjectBoundary_loss_preprocess.tif')

### the last line of code will save a new GFW layer that has the same CRS and extent of your specific property
### take note of where you save this layer! You can create a specific folder like I did to store it (GFW is the new folder I created)

In [ ]:
# STEP 3 - Calculation

# open newly created GFW loss file of that specific property (ex. ProjectBoundary_loss_preprocess.tif) 
GFW_loss_file = os.path.join(filepath, 'LossPreprocess/ProjectBoundary_loss_preprocess.tif') ## CHANGE to name of your file 
GFW_loss = rioxarray.open_rasterio(GFW_loss_file)
GFW_loss = GFW_loss.where(GFW_loss >= -100)
GFW_loss += 2000

# open 2017-2020 carbon stock files
filepath_post = '/content/drive/my_file_path/AGB_Years/Post2016' #change to exact version of filepath
data = glob.glob(os.path.join(filepath_post, '*.tif')) 

for i, data_path in enumerate(data):
  raster = rioxarray.open_rasterio(data_path).astype(np.float32)
  raster = raster.where(raster >= -100)
  year = int(os.path.splitext(os.path.split(data_path)[1])[0][5:13])
  loss_pixels = (GFW_loss == year)
  stock_loss_year = raster.where(loss_pixels).sum().data*900/1e3
  print(year, ':', stock_loss_year) 

In [ ]:
### See ED_Version Comparison sheet for 2017-2020 loss values for answer key 